# **cookbook** | webscraping

Data extraction from webpage: https://uwielbiam.pl

In [1]:
import re
import numpy as np
import pandas as pd
import httplib2
import requests
from bs4 import BeautifulSoup, SoupStrainer
from collections import Counter
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

### MAIN DISHES

In [2]:
class Webpage(object):
    
    def __init__(self, link):
        self.link = link
        self.link_przepisy = link + '/przepisy?pg='
        self.link_przepis = link + '/przepis/'
        
    def one_page_meals(self, number):
        """Return list with all dishes from a particular page"""
        if number > 0:
            all_links = []
            http = httplib2.Http()
            status, response = http.request(self.link_przepisy + str(number))
            for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                if link.has_attr('href') and link['href'].startswith('/przepis/'):
                    all_links.append(((link['href'])[len('/przepis/'):]).replace('-', ' '))
            return all_links
        else:
            print("number variable needs to be greater than zero.")

    def one_page_links(self, number):
        """Return list with all dishes links from a particular page"""
        if number > 0:
            all_links = []
            http = httplib2.Http()
            status, response = http.request(self.link_przepisy + str(number))
            for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                if link.has_attr('href') and link['href'].startswith('/przepis/'):
                    all_links.append(self.link_przepis + (link['href'])[len('/przepis/'):])
            return all_links
        else:
            print("number variable needs to be greater than zero.")
            
    @property
    def all_links(self):
        """Return list with all dishes names from the webpage"""
        all_pages = 458
        if number > 0:
            all_links = []
            for i in range(1, all_pages):
                if i%10 == 0:
                    print(f'{i}/{all_pages}', end='\r')
                http = httplib2.Http()
                status, response = http.request(self.link_przepisy + str(i))
                for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                    if link.has_attr('href') and link['href'].startswith('/przepis/'):
                        all_links.append(self.link_przepis + (link['href'])[len('/przepis/'):])
            return all_links
        else:
            print("number variable needs to be greater than zero.")

In [3]:
uwielbiam = Webpage('http://uwielbiam.pl/')
uwielbiam.link

'http://uwielbiam.pl/'

In [4]:
uwielbiam.one_page_meals(1)

['salatka z brukselka i ogorkami kiszonymi krakus',
 'salatka z gruszka i serem plesniowym',
 'salatka z winogronem i serem plesniowym',
 'salatka z figami i szynka parmenska',
 'salatka z winogronem i kurczakiem',
 'salatka ziemniaczana z bobem boczkiem i korniszonami z chili i francuskim ',
 'salatka z pieczonych burakow i mozzarelli z dressingiem z syropu malinowego 2',
 'mini makaron czekoladowy z truskawkami arbuzem i sosem czekoladowym',
 'salatka z makaronem czekoladowym feta arbuzem zielonymi oliwkami i cebula',
 'wiosenna salatka z kalarepy z ogorkami konserwowymi krakus',
 'salatka ziemniaczana z ogorkami kiszonymi krakus',
 'salatka gyros ale grecka z ogorkami korniszony z chili',
 'salatka makaronowa z pieczonym miesem',
 'makaron ryzowy z awokado jajkiem mozzarella i sosem bazyliowym',
 'salatka jajeczna z paluszkami krabowymi',
 'salatka z jarmuzu i awokado',
 'kasza peczak ze szpinakiem',
 'salatka z awokado i lososiem',
 'salatka z granatem i kurczakiem',
 'salatka jarz

In [5]:
uwielbiam.one_page_links(1)

['http://uwielbiam.pl//przepis/salatka-z-brukselka-i-ogorkami-kiszonymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-z-gruszka-i-serem-plesniowym',
 'http://uwielbiam.pl//przepis/salatka-z-winogronem-i-serem-plesniowym',
 'http://uwielbiam.pl//przepis/salatka-z-figami-i-szynka-parmenska',
 'http://uwielbiam.pl//przepis/salatka-z-winogronem-i-kurczakiem',
 'http://uwielbiam.pl//przepis/salatka-ziemniaczana-z-bobem-boczkiem-i-korniszonami-z-chili-i-francuskim-',
 'http://uwielbiam.pl//przepis/salatka-z-pieczonych-burakow-i-mozzarelli-z-dressingiem-z-syropu-malinowego-2',
 'http://uwielbiam.pl//przepis/mini-makaron-czekoladowy-z-truskawkami-arbuzem-i-sosem-czekoladowym',
 'http://uwielbiam.pl//przepis/salatka-z-makaronem-czekoladowym-feta-arbuzem-zielonymi-oliwkami-i-cebula',
 'http://uwielbiam.pl//przepis/wiosenna-salatka-z-kalarepy-z-ogorkami-konserwowymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-ziemniaczana-z-ogorkami-kiszonymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-

### DISH PROPERTIES

In [6]:
class Uwielbiam_dish_page(object):
    
    def __init__(self, link):
        self.link = link
        self.html_content = BeautifulSoup(requests.get(link).content)
        
    @property
    def title(self):
        title_content = self.html_content.find('h1', attrs = {'class': 'h-section'})
        return title_content.text
    
    @property
    def description(self):
        description_content = self.html_content.find('p', attrs = {'class': 'p--description'})
        try:
            description_content = description_content.text.replace('\u2028\r', ' ').replace('  \n', ' ').replace('\r\n', '')
        except:
            description_content = 'None'
        return description_content
    
    @property
    def ingredients(self):
        ingr = self.html_content.find_all('tr', attrs = {'class': 'ingredients_row'})
        x_list = []
        for x in ingr:
            x_list.append(x.text.split())
        ingredients = []
        for x in x_list[:-2]:
            try: 
                x1 = int(x[0])
                x2 = x[1]
                x3 = ' '.join(x[2:])
                ingredients.append([x1, x2, x3])
            except:
                ingredients.append([' '.join(x)])
            pass
        return ingredients
    
    @property
    def preparation(self):
        preparation_content = self.html_content.find_all('li', attrs = {'class': 'recipe-list--row'})
        prep_content = []
        for i in range(len(preparation_content)):
            prep_content.append(preparation_content[i].text.strip().replace('\r\n', ''))
        return prep_content
    
    @property
    def tags(self):
        tags = self.html_content.find_all('a', attrs = {'class': 'tag f-element'})
        tag_content = []
        for n in range(len(tags)):
            tag_content.append(tags[n].text.lower())
        return tag_content
    
    @property
    def allergens(self):
        ingr = self.html_content.find_all('tr', attrs = {'class': 'ingredients_row'})
        x_list = []
        for x in ingr:
            x_list.append(x.text.split())
        save = 0
        allergens = []
        for counter, item in enumerate(x_list[-2]):
            if item == "Alergeny":
                save = 1
            else:
                if save == 1:
                    allergens.append(item)
                else:
                    pass
        if allergens == []:
            return 'None'
        return allergens
    
    @property
    def skill(self):
        return self.html_content.find_all('span', attrs = {'class': 'ico-info_content'})[0].text
        
    @property
    def time(self):
        return self.html_content.find_all('span', attrs = {'class': 'ico-info_content'})[1].text
    
    @property
    def author(self):
        return self.html_content.find_all('p', attrs = {'class': 'user_name'})[1].text

In [7]:
meal = Uwielbiam_dish_page('https://uwielbiam.pl/przepis/ciasteczka-thumbprint-z-nadzieniem-krowkowym-i-orzechami')
meal.link

'https://uwielbiam.pl/przepis/ciasteczka-thumbprint-z-nadzieniem-krowkowym-i-orzechami'

In [8]:
meal.title

'Ciasteczka thumbprint z nadzieniem krówkowym i orzechami'

In [9]:
meal.description

'Przedstawiamy naszą krówkowo-orzechową ciasteczkową wariację. Są to pyszne, chrupiące i bardzo efektowne ciasteczka z wyraźną nutą kakaową. Idealnie sprawdzą się jako dodatek do kawy lub kubka herbaty podczas długich jesienno-zimowych wieczorów.'

In [10]:
meal.ingredients

[['SKŁADNIKI NA CIASTO'],
 [130, 'Gramów', 'mąka pszenna tortowa Lubella'],
 [30, 'Gramów', 'Kakao DecoMorreno'],
 [110, 'Gramów', 'masło'],
 [150, 'Gramów', 'cukier'],
 [1, 'Szczypta', 'sól'],
 [2, 'Sztuki', 'jajko'],
 [1, 'Szklanka', 'Orzechy laskowe mielone'],
 ['SKŁADNIKI NA NADZIENIE'],
 [150, 'Gramów', 'Cukierki krówki'],
 [3, 'Łyżki', 'śmietanka kremowa 30%'],
 [50, 'Gramów', 'Mleczna czekolada']]

In [11]:
meal.preparation

['Wymieszaj w misce mąkę i kakao. W drugiej misce zmiksuj masło z cukrem i solą na puszystą masę. Dodaj żółtka z jajek. W następnej kolejności dodaj suche składniki i miksuj całość aż do połączenia.',
 'Przykryj ciasto folią i wstaw do lodówki na godzinę. Następnie roztrzep białko widelcem. Formuj małe kulki z ciasta, mocz je w białku i obtaczaj w orzechach.',
 'Przygotowane w ten sposób ciastka układaj na blasze w odstępach. Na środku każdego ciasteczka zrób wgłębienie. Piecz je w  nagrzanym do 180°C piekarniku przez około 10-12 minut.',
 'Upieczone ciastka ostudź, po czym ponownie zrób w nich wgłębienia. W małym rondlu rozpuść krówki ze śmietanką. Tak przygotowaną masą wypełnij wgłębienia w ciastkach.',
 'Gdy masa krówkowa zastygnie, rozpuść w kąpieli wodnej czekoladę i polej nią ciastka.']

In [12]:
meal.tags

['sól',
 'na słodko',
 'cukier',
 'masło',
 'wypieki',
 'impreza',
 'deser',
 'kakao decomorreno',
 'jajko',
 'podwieczorek',
 'kakao',
 'przystawki i przekąski',
 'mąka pszenna tortowa lubella',
 'bezmięsna',
 'śmietanka kremowa 30%',
 'urodziny imieniny',
 'grill i piknik',
 'mleczna czekolada',
 'orzechy laskowe mielone',
 'cukierki krówki']

In [13]:
meal.allergens

['Gluten', 'Laktoza', 'Mleko', 'Jajka']

In [14]:
meal.skill

'Średni'

In [15]:
meal.time

'Do 1,5 godz'

In [16]:
meal.author

'decomorreno'

### DATA EXTRACTION

In [17]:
def dish_information(link):
    """Return list with information about the meal."""
    meal = Uwielbiam_dish_page(link)
    return [meal.title, meal.description, meal.ingredients, meal.preparation, meal.tags,
           meal.allergens, meal.skill, meal.time, meal.author, meal.link]

In [18]:
def extract_data(main_link, save_sample_data=False):
    """
    Function which takes as a input list with dishes links and return
    a DataFrame with dish: title, description, ingredients, preparation procedure, 
    tags and allergens.
    """
    
    # build object of the webpage
    uwielbiam = Webpage(main_link)
    
    # get list with meal links from page: 5
    # (change to: uwielbiam.all_links if you want to get all data)  
    links = uwielbiam.one_page_links(5)  
    
    # extract meal information from each page
    cookbook_dict = {}
    for counter in tqdm(range(len(links))):
        cookbook_dict[counter] = dish_information(links[counter])

    column_names = ["title", "description", "ingredients", "preparation", "tags", "allergens", "skill", "time", "author", "link"]
    cookbook = pd.DataFrame.from_dict(cookbook_dict, orient='index', columns=column_names)

    # save data to .csv file
    if save_sample_data == True: cookbook.to_csv('cookbook_sample.csv')

    return cookbook

In [19]:
DATA = extract_data(uwielbiam.link, True)
DATA

100%|██████████| 20/20 [00:37<00:00,  1.89s/it]


,title,description,ingredients,preparation,tags,allergens,skill,time,author,link
0,Krem czekoladowy z nutą korzenną,"Oto ekspresowy pomysł na idealny, bardzo odżyw...","[[3, Łyżki, Kakao DecoMorreno], [500, Gramów, ...",[Na suchej patelni podpraż łuskane pestki słon...,"[dla rodziny, sól, na słodko, kakao decomorren...",None,Łatwy,Do 15 min,decomorreno,http://uwielbiam.pl//przepis/krem-czekoladowy-...
1,Ciasteczkowe choinki,Bajkowo prezentujące się kremowe choinki to ni...,"[[SKŁADNIKI NA CIASTO], [25, Gramów, Kakao Dec...",[Wszystkie składniki na ciasto zagnieć szybko....,"[na słodko, masło, dla dzieci, deser, kakao de...","[Gluten, Laktoza, Mleko, Jajka]",Łatwy,Do 1 godz,decomorreno,http://uwielbiam.pl//przepis/ciasteczkowe-choinki
2,Tort czekoladowy z kremem cynamonowym,"Poznaj naszą tortową propozycję super food, kt...","[[SKŁADNIKI SUCHE], [120, Gramów, Kakao DecoMo...",[W misce wymieszaj wszystkie suche składniki. ...,"[sól, na słodko, wypieki, woda, deser, kakao d...",[Jajka],Średni,Do 1 godz,decomorreno,http://uwielbiam.pl//przepis/tort-czekoladowy-...
3,Ciasto barbórkowe z galaretką,Zainspiruj się naszym przepisem na bardzo lekk...,"[[130, Gramów, mąka pszenna tortowa Lubella], ...",[Białka oddziel od żółtek i ubij na sztywną pi...,"[na słodko, cukier, wypieki, impreza, deser, k...","[Gluten, Jajka]",Średni,Do 1 godz,decomorreno,http://uwielbiam.pl//przepis/ciasto-barborkowe...
4,Paszteciki z grzybami i kapustą,None,"[[CIASTO], [350, Gramów, mąka Lubella Puszysta...",[Najpierw przygotować farsz. Grzyby zalać wrzą...,"[polska, obiad, dania główne, sól, cukier, ceb...",None,Łatwy,Ponad 3 godz,lubella,http://uwielbiam.pl//przepis/paszteciki-z-grzy...
5,Sernik królewski,None,"[[KRUCHE CIASTO], [300, Gramów, Lubella Mąka u...","[Mąkę, kakao, proszek i cukier wymieszać, zrob...","[polska, sól, na słodko, cukier, jaja kurze ca...",None,Łatwy,Do 2 godz,lubella,http://uwielbiam.pl//przepis/sernik-krolewski-3
6,Pierniczki świąteczne z lukrem królewskim,None,"[[CIASTO], [500, Gramów, mąka Lubella Puszysta...",[Miód oraz masło przełożyć do rondelka i rozpu...,"[polska, na słodko, jaja kurze całe, wypieki, ...",None,Średni,Do 3 godz,lubella,http://uwielbiam.pl//przepis/pierniczki-swiate...
7,Uszka z grzybami,None,"[[CIASTO], [300, Gramów, mąka Lubella Puszysta...","[Mąkę przesiać do miski, połączyć z solą, wodą...","[polska, obiad, dla rodziny, dania główne, sól...",None,Łatwy,Do 3 godz,lubella,http://uwielbiam.pl//przepis/uszka-z-grzybami
8,Tradycyjne pierogi z kapusta i grzybami,None,"[[CIASTO:], [350, Gramów, mąka Lubella Puszyst...","[Mąkę przesiać do miski, połączyć z solą, wodą...","[polska, obiad, dla rodziny, dania główne, sól...",None,Łatwy,Do 3 godz,lubella,http://uwielbiam.pl//przepis/tradycyjne-pierog...
9,Czekoladowo - miętowa mokka,Czekolada i mięta to dwa niezwykle wyraziste s...,"[[1, Łyżka, Kakao DecoMorreno], [60, Mililitró...",[W rondelku podgrzej mleko wraz z kakao oraz k...,"[na słodko, deser, kakao decomorreno, śniadani...",None,Łatwy,Do 15 min,decomorreno,http://uwielbiam.pl//przepis/czekoladowo-mieto...
